# Final Project - Fundamentals of Deep Learning for NLP and CV

Congratulations! This is the final project!


## Delivery of Project

This jupyter notebook is to be delivered to evaluate your knowledge on the Deep Learning for NLP and CV module at Rumos, before date agreed with the professor. Please add your name and e-mail next.

**Student Name**: "Nuno Pereira"  
**E-mail**: "pereiranuno88@gmail.com"


## Instructions




## Details on the dataset





## Plagiarism

Always remember that you are here to learn. Discussions on the final project are highly incentivised but please do not share your work. The struggle to solve the problems is needed in order to become a true Data Scientist. By allowing others to use your code you are making the world a worse place: you are not truly helping your colleague, and you are not promoting discussions on the topic.

In case you need help, or just want to discuss some project-related topics, reach out to me either through email or through a Slack direct message.

# Objectives

Please solve the following exercises by creating a markdown cell with **# EXERCISE >>NUMBER<<**  just before you solve it (you can use the number of cells you need after that).

## Evaluation
Points (of a total of 100%):
1. 20%  
2. 20%  
3. 20%  
4. 20%  
5. 20%  

Final 5% for additional effort and conclusions beyond what was asked (give your _extra mile_).

## Important notes
1. Data Science is all about *flow*. Keep your analysis work-flow consistent.  
2. When it is requested you to *describe* something, please be 1. skeptic, 2. objective, and 3. succinct! 
3. If you don't know: search, invent, study, but please don't leave any exercise blank.

### Good luck!
# 3, 2, 1, GO! GO! GO!

### Import libraries

In [1]:
import pandas as pd

import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# Global variables for reproducibility
random_seed = 42
torch.manual_seed(random_seed)

### Load Data

### CIFAR-10Dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class. 

<img src="images/cifar10.png" width="400" height="100">

[CIFAR 10](https://www.cs.toronto.edu/~kriz/cifar.html)

In [3]:
transform = transforms.Compose(
    [
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Twitter Dataset

[Source - huggingface.co/datasets - carblacac/twitter-sentiment-analysis](https://huggingface.co/datasets/carblacac/twitter-sentiment-analysis)

In [4]:
twitter_df = pd.read_csv("data/twitter.csv", sep="\t", header=None, names=["target", "text"])
twitter_df.head()

,target,text
0,"target,text",NaN
1,"0,Starting back at work today Looks like it...",NaN
2,"1,Sugar levels dropping... munchies setting in...",NaN
3,"1,@karineb22 yeah!!! have a great summer break!",NaN
4,"1,hannah montana was very good. now going to ...",NaN


In [5]:
twitter_df.shape

(149986, 2)

# EXERCISE 1 - Use CIFAR10 Dataset

```classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')```

- Build a simple Neural Network without using convolutional layers to predict the image class
    - No need to configure the optimization, loss function or predict yet. Only implement the NN architecture as ```class NeuralNetwork(nn.Module)```
- Explain your choices for the model architecture e.g., activation layer, input and output

---

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Primeira camada 3 canais (RGB) *  32x32 pixels : 3072 → 768 neurónios
        self.hidden1 = nn.Linear(3*32*32, 768)
        
        # Segunda camada oculta: 768 → 384 neurónios
        self.hidden2 = nn.Linear(768, 384)
        
        # Terceira camada oculta: 384 → 192 neurónios
        self.hidden3 = nn.Linear(384, 192)
        
        # Camada de saída: 192 → 10 neurónios (correspondentes às 10 classes do CIFAR-10)
        self.out = nn.Linear(192, 10)
        
        # Dropout com probabilidade de 20% para evitar overfitting
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Achatar a imagem (flatten) para vetor 1D, ada imagem passa a ser um vetor com 3072 números, pronto para entrar numa camada Linear.
        x=x.view(-1,32*32*3)
        
        # Passagem pela primeira camada com ReLU e Dropout
        x = F.relu(self.hidden1(x))
        x = self.dropout(x)
        
        # Segunda camada com ReLU e Dropout
        x = F.relu(self.hidden2(x))
        x = self.dropout(x)
        
        # Terceira camada com ReLU
        x = F.relu(self.hidden3(x))
        
        x = self.out(x)
        
        return x


A rede neural construída é composta por três camadas ocultas e uma camada de saída, desenvolvida para classificar imagens do conjunto de dados CIFAR-10. Como as imagens têm dimensão 32x32 píxeis com 3 canais (RGB), cada imagem é composta por um total de 3072 píxeis. Antes de ser processada pela rede, cada imagem é achatada e  transformando-se num vetor unidimensional com 3072 elementos, o que permite a sua passagem por camadas lineares.

A primeira camada oculta da rede transforma este vetor de 3072 valores num vetor de 768 neurónios, ao qual é aplicada a função de ativação ReLU para introduzir não-linearidade. Em seguida, é aplicado um Dropout de 20%, que desativa aleatoriamente uma parte dos neurónios durante o treino, reduzindo o risco de overfitting. Esta combinação (Linear → ReLU → Dropout) repete-se na segunda camada oculta (768 → 384 neurónios) e na terceira camada oculta (384 → 192 neurónios), mantendo a regularização e promovendo uma redução progressiva da dimensionalidade.

Por fim, a camada de saída converte os 192 neurónios da última camada oculta num vetor de 10 saídas, correspondentes às 10 classes do CIFAR-10 ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck').

---

# Exercise 2 - Use CIFAR10 Dataset
- Set the optimizer, loss function and train your model
    - Explain your choices for the optimizer and loss function
- Check the performance of your model
    - Chose the metric and explain your choice

---

In [ ]:
# Instanciar o modelo
model = NeuralNetwork()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Definir a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Nesta secção, é feita a preparação do modelo para o processo de treino.

Primeiramente, a classe NeuralNetwork é instanciada, criando-se um objeto chamado model que representa a arquitetura da rede neural definida anteriormente.

De seguida, é determinado o dispositivo onde o modelo será treinado. Utiliza-se a função torch.device("cuda" if torch.cuda.is_available() else "cpu"), que verifica automaticamente se existe uma placa gráfica (GPU) compatível com CUDA disponível. Se existir, o modelo será treinado na GPU, o que acelera significativamente o processo; caso contrário, o treino decorre no processador (CPU).


Depois, define-se a função de perda (loss function), que neste caso é a CrossEntropyLoss. Esta função foi escolhida tendo em conta que temos um problema de classificação multiclasse, como é o caso do CIFAR-10.

Por fim, é criado o otimizador, que neste caso é o Adam. É inicializado com os parâmetros do modelo (model.parameters()) e com uma taxa de aprendizagem (learning rate) de 0.001, que controla o quão rapidamente os pesos da rede são ajustados durante o treino.

In [36]:
def train_model(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0

    for batch_idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    return average_loss

A função train_one_epoch  recebe como argumentos o modelo, o dataloader (que fornece os dados em lotes), o otimizador, a função de perda e o dispositivo onde os cálculos devem ser realizados (CPU ou GPU).

O primeiro passo é colocar o modelo em modo de treino através de model.train(). Este modo ativa componentes como o Dropout, caso existam, garantindo que o comportamento do modelo é o apropriado para a fase de aprendizagem.

De seguida, é inicializada uma variável total_loss que irá acumular o valor da perda (erro) ao longo de todos os lotes processados.

O ciclo principal da função percorre o dataloader, recebendo os dados (inputs) e os rótulos correspondentes (targets) em cada iteração. 

As previsões são comparadas com as labels reais através da função de perda (loss_fn), que calcula o erro cometido.

Antes de atualizar os pesos do modelo, é necessário limpar os gradientes anteriores com optimizer.zero_grad(). 
Depois disso, realiza-se a propagação para trás (backward pass), onde os gradientes do erro em relação aos pesos são calculados com loss.backward(). Finalmente, o otimizador atualiza os pesos da rede com optimizer.step().


No final  a função calcula a perda média dividindo a perda acumulada pelo número total de lotes, e devolve esse valor.

In [37]:
def evaluate_model(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            total_loss += loss.item()

            predictions = outputs.argmax(dim=1)
            total_correct += (predictions == targets).sum().item()
            total_samples += targets.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = 100 * total_correct / total_samples
    return avg_loss, accuracy


A função evaluate_model serve para avaliar o desempenho do modelo sobre o conjunto de dados. Esta função recebe como argumentos o modelo a avaliar, o dataloader com os dados a testar, a função de perda utilizada e o dispositivo onde os cálculos devem ser realizados (CPU ou GPU).

O  modelo é colocado em modo de avaliação com model.eval(). Esta instrução garante que o modelo se comporta corretamente nesta fase, desativando por exemplo camadas como o Dropout, que só são úteis durante o treino.

São depois inicializadas três variáveis: total_loss, para acumular o valor total da perda; total_correct, que conta quantas previsões foram corretas; e total_samples, que regista o número total de amostras avaliadas.

O bloco with torch.no_grad() é utilizado para garantir que o PyTorch não calcula nem guarda gradientes durante esta fase, o que melhora o desempenho e reduz o uso de memória, já que não há necessidade de retropropagação nesta etapa.

O ciclo for percorre todos os lotes de dados no dataloader. Para cada lote, os dados (inputs) e os rótulos verdadeiros (targets) são enviados para o dispositivo apropriado (GPU ou CPU).


As previsões são depois convertidas nas classes previstas pelo modelo, utilizando argmax(dim=1), que seleciona o índice com maior valor de saída (ou seja, a classe mais provável). A função compara estas previsões com os rótulos verdadeiros para contar quantas previsões foram corretas, incrementando total_correct. O número de amostras do lote é igualmente somado ao contador total_samples.

Após o ciclo, é calculada a perda média, dividindo o valor total da perda pelo número de lotes (len(dataloader)). A métrica Accuracy  é calculada como a percentagem de previsões corretas em relação ao total de amostras avaliadas.


In [38]:
epochs = 10
for epoch in range(epochs):
    print(f"epoch {epoch + 1}/{epochs}")

    train_loss = train_model(model, trainloader, optimizer, criterion, device)
    val_loss, val_acc = evaluate_model(model, testloader, criterion, device)

    print(f"  → Loss Train: {train_loss:.4f}")
    print(f"  → Loss Validation: {val_loss:.4f} | Accuracy: {val_acc:.2f}%\n")

print("Treino concluído!")



epoch 1/10
  → Loss Train: 0.8199
  → Loss Validation: 1.4114 | Accuracy: 54.53%

epoch 2/10
  → Loss Train: 0.8033
  → Loss Validation: 1.4268 | Accuracy: 55.25%

epoch 3/10
  → Loss Train: 0.7868
  → Loss Validation: 1.4165 | Accuracy: 55.05%

epoch 4/10
  → Loss Train: 0.7746
  → Loss Validation: 1.4473 | Accuracy: 55.40%

epoch 5/10
  → Loss Train: 0.7564
  → Loss Validation: 1.4526 | Accuracy: 55.08%

epoch 6/10
  → Loss Train: 0.7428
  → Loss Validation: 1.4283 | Accuracy: 55.89%

epoch 7/10
  → Loss Train: 0.7312
  → Loss Validation: 1.4531 | Accuracy: 55.67%

epoch 8/10
  → Loss Train: 0.7177
  → Loss Validation: 1.4505 | Accuracy: 55.78%

epoch 9/10
  → Loss Train: 0.6994
  → Loss Validation: 1.5303 | Accuracy: 54.71%

epoch 10/10
  → Loss Train: 0.6946
  → Loss Validation: 1.4731 | Accuracy: 55.75%

Treino concluído!


A variável epochs define o número total de iterações a realizar. O ciclo for itera esse número de vezes, controlando a progressão do treino. A cada iteração , imprime-se no ecrã o número da época atual, no formato epoch X/Y, para que o utilizador acompanhe o progresso.

Durante cada iteração, são chamadas duas funções principais:

   -train_model(...): esta função realiza o treino do modelo com base nos dados de treino (trainloader).

   -evaluate_model(...): após o treino, esta função avalia o desempenho do modelo no conjunto de teste (testloader).

Ao longo das iterações  observa-se uma redução progressiva da perda de treino, o que indica que o modelo está a conseguir ajustar-se aos dados de treino e a aprender padrões. A perda de treino começa em 0.82 na primeira iteração e desce para 0.69 na última, o que é um comportamento esperado e positivo.

No entanto, a perda de validação mantém-se praticamente estável e relativamente alta, a oscilar entre 1.41 e 1.53, sem mostrar sinais claros de melhoria. Isto, aliado a uma Accuracy  também estável, entre 54.5% e 55.9%, sugere que o modelo atingiu um patamar de desempenho limitado com a arquitetura atual e que pode estar a começar a sobreajustar-se (overfitting) aos dados de treino.


O modelo está a aprender durante o treino, mas não está a generalizar suficientemente bem para os dados de teste. A arquitetura e os hiperparâmetros permitem um desempenho razoável, com uma accuracy próxima dos 55%, o que está alinhado com redes densas simples (sem camada de convolução). 

---

# Exercise 3 - Use CIFAR10 Dataset
- Same as Exercise 1 but now add Convolutional Layers
- Explain your choices for the model architecture e.g., activation layer, input and output

# Exercise 4 - Use CIFAR10 Dataset
- Same as Exercise 2 but for the CNN arquitecture (model from Exercise 3)

- Set the optimizer, loss function and train your model
    - Explain your choices for the optimizer and loss function
- Check the performance of your model
    - Chose the metric and explain your choice

# Exercise 5 - Use Twitter Dataset
For the **Text DATA TODO**

- Build a Supervised Classification model
- Explain your choices for preparing the text for the model